In [1]:
import pandas as pd

In [71]:
df1 = pd.read_csv('Entry and Exit data.csv')

In [72]:
df2 = pd.read_csv('2018_more.csv')

In [73]:
df2['Date']=pd.to_datetime(df2['Dates[Travel Date (dd/mm/yyyy)]'])

C:\Users\alastair.tyson\AppData\Local\Temp\ipykernel_9256\1311326412.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df2['Date']=pd.to_datetime(df2['Dates[Travel Date (dd/mm/yyyy)]'])


In [74]:
df2['Month']=df2['Date'].apply(lambda x: x.month_name())
df2['Year']=df2['Date'].apply(lambda x: x.year)

In [75]:
df2

,Stations[Station],Stations[NLC],Dates[Travel Date (dd/mm/yyyy)],Entry / Exit[Entry / Exit],[SumTap_Count],Date,Month,Year
0,Abbey Road DLR,750,01/06/2019,Entry,868,2019-01-06,January,2019
1,Abbey Road DLR,750,01/06/2019,Exit,656,2019-01-06,January,2019
2,Abbey Road DLR,750,02/06/2019,Entry,490,2019-02-06,February,2019
3,Abbey Road DLR,750,02/06/2019,Exit,470,2019-02-06,February,2019
4,Abbey Road DLR,750,03/06/2019,Entry,969,2019-03-06,March,2019
...,...,...,...,...,...,...,...,...
407385,Woolwich EL,6562,28/09/2022,Exit,14437,2022-09-28,September,2022
407386,Woolwich EL,6562,29/09/2022,Entry,15384,2022-09-29,September,2022
407387,Woolwich EL,6562,29/09/2022,Exit,14476,2022-09-29,September,2022
407388,Woolwich EL,6562,30/09/2022,Entry,14137,2022-09-30,September,2022


In [77]:
out = df2.groupby(['Stations[Station]','Year','Month','Entry / Exit[Entry / Exit]']).sum(numeric_only = True).reset_index()

In [79]:
out = out.drop(['Stations[NLC]'], axis = 1)

In [88]:
out.head()

,station_name,year,month,type,taps
0,Abbey Road DLR,2019,April,Entry,3382
1,Abbey Road DLR,2019,April,Exit,3237
2,Abbey Road DLR,2019,August,Entry,17762
3,Abbey Road DLR,2019,August,Exit,16836
4,Abbey Road DLR,2019,December,Entry,3764


In [81]:
out.columns = ['station_name', 'year', 'month', 'type', 'taps']

In [89]:
month_no = {'January' : '01', 'February' : '02', 'March' : '03', 'April' : '04', 'May' : '05', 'June' : '06',
           'July' : '07', 'August': '08', 'September' : '09', 'October' : '10', 'November' : '11', 'December' : '12'}

In [117]:
name = []
date = []
typee = []
taps = []

for i in range(len(out)):
    name.append(out.iloc[i,0])
    typee.append(out.iloc[i,3])
    taps.append(out.iloc[i,4])
    dat = ''
    if out.iloc[i,2] in ['April', 'June', 'September', 'November']:
        dat += '30/{}/'.format(month_no[out.iloc[i,2]])
    elif out.iloc[i,2] in ['January', 'March', 'May', 'July', 'August', 'October', 'December']:
        dat += '31/{}/'.format(month_no[out.iloc[i,2]])
    else:
        if out.iloc[i, 1] != 2020:
            dat += '28/{}/'.format(month_no[out.iloc[i,2]])
            
        else:
            dat += '29/{}/'.format(month_no[out.iloc[i,2]])
    dat += str(out.iloc[i,1])
    date.append(pd.to_datetime(dat, format ='%d/%m/%Y'))

In [118]:
out2 = pd.DataFrame({'station_name' : name, 'date' : date, 'type' : typee, 'taps' : taps})

In [119]:
out2 = out2.sort_values(by=['station_name','date']).reset_index(drop = True)

In [120]:
out2

,station_name,date,type,taps
0,Abbey Road DLR,2019-01-31,Entry,3268
1,Abbey Road DLR,2019-01-31,Exit,2949
2,Abbey Road DLR,2019-02-28,Entry,3256
3,Abbey Road DLR,2019-02-28,Exit,3055
4,Abbey Road DLR,2019-03-31,Entry,3485
...,...,...,...,...
40563,Woolwich EL,2022-10-31,Exit,32827
40564,Woolwich EL,2022-11-30,Entry,23549
40565,Woolwich EL,2022-11-30,Exit,22992
40566,Woolwich EL,2022-12-31,Entry,34550


In [122]:
name = []
date = []
typee = []
taps = []
for i in range(len(df1)):
    
    dat = ''
    if df1.iloc[i,1] in ['April', 'June', 'September', 'November']:
        dat += '30/{}/'.format(month_no[df1.iloc[i,1]])
    elif df1.iloc[i,1] in ['January', 'March', 'May', 'July', 'August', 'October', 'December']:
        dat += '31/{}/'.format(month_no[df1.iloc[i,1]])
    else:
        dat += '28/{}/'.format(month_no[df1.iloc[i,1]])
    dat += str(df1.iloc[i,2])
    dat = pd.to_datetime(dat, format ='%d/%m/%Y')
    if dat < pd.to_datetime('01/01/2019', format ='%d/%m/%Y'):
        date.append(dat)
        name.append(df1.iloc[i,0])
        typee.append(df1.iloc[i,3])
        taps.append(df1.iloc[i,4])

In [123]:
out3 = pd.DataFrame({'station_name' : name, 'date' : date, 'type' : typee, 'taps' : taps})

In [124]:
out3 = out3.sort_values(by=['station_name','date']).reset_index(drop = True)

In [125]:
out3

,station_name,date,type,taps
0,Abbey Road DLR,2018-04-30,Entry,18151
1,Abbey Road DLR,2018-04-30,Exit,15674
2,Abbey Road DLR,2018-05-31,Exit,17534
3,Abbey Road DLR,2018-05-31,Entry,20034
4,Abbey Road DLR,2018-06-30,Exit,16897
...,...,...,...,...
11973,Worcester Park,2018-10-31,Entry,47776
11974,Worcester Park,2018-11-30,Entry,44116
11975,Worcester Park,2018-11-30,Exit,40426
11976,Worcester Park,2018-12-31,Entry,37169


In [138]:
out4 = pd.concat([out3,out2])

In [84]:
df1 = df1.drop(['Travel Location No'], axis = 1)

In [85]:
df1.columns = ['station_name', 'year', 'month', 'type', 'taps']

In [86]:
df1

,station_name,year,month,type,taps
0,Abbey Road DLR,April,2018,Entry,18151
1,Abbey Road DLR,April,2018,Exit,15674
2,Abbey Wood,April,2018,Exit,109562
3,Abbey Wood,April,2018,Entry,137627
4,Acton Central,April,2018,Exit,70946
...,...,...,...,...,...
47401,Woolwich Arsenal NR,January,2021,Exit,35959
47402,Woolwich Dockyard,January,2021,Exit,1888
47403,Woolwich Dockyard,January,2021,Entry,2639
47404,Worcester Park,January,2021,Entry,4395


In [139]:
station_name = []
station_type = []

for i in range(len(out4)):
    name = out4.iloc[i,0]
    if 'DLR' in name:
        station_type.append('DLR')
        station_name.append(name.replace(' DLR',''))
    elif 'EL' in name:
        station_type.append('EL')
        station_name.append(name.replace(' EL',''))
    elif 'NR' in name:
        station_type.append('NR')
        station_name.append(name.replace(' NR',''))
    elif 'TRAM' in name:
        station_type.append('TRAM')
        station_name.append(name)
    elif 'LU' in name:
        station_type.append('LU')  
        station_name.append(name.replace(' LU',''))
    elif 'LO' in name:
        station_type.append('LO')
        station_name.append(name.replace(' LO',''))
    else:
        station_type.append('check')
        station_name.append(name)

In [140]:
out4['station_name'] = station_name
out4['station_type'] = station_type

In [143]:
out4.to_csv('tfl.csv')

In [141]:
out4

,station_name,date,type,taps,station_type
0,Abbey Road,2018-04-30,Entry,18151,DLR
1,Abbey Road,2018-04-30,Exit,15674,DLR
2,Abbey Road,2018-05-31,Exit,17534,DLR
3,Abbey Road,2018-05-31,Entry,20034,DLR
4,Abbey Road,2018-06-30,Exit,16897,DLR
...,...,...,...,...,...
40563,Woolwich,2022-10-31,Exit,32827,EL
40564,Woolwich,2022-11-30,Entry,23549,EL
40565,Woolwich,2022-11-30,Exit,22992,EL
40566,Woolwich,2022-12-31,Entry,34550,EL


In [ ]:
station_type=[]

for i in out4.station_name.unique()